In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# load demand data
df_demand= pd.read_csv('end_use_demand_model/POPadjusted_demand_00to10.csv')
df_demand['AEST']=pd.to_datetime(df_demand['AEST'])
df_demand=df_demand.set_index('AEST')
df_demand=df_demand.loc[df_demand.index.year<=2009] # drop 2010
df_demand

,NSW,QLD,VIC,SA,TAS,NEM
AEST,,,,,,
2000-01-01 00:00:00,8329.481952,5726.432366,6144.580476,1611.002446,1066.124437,22877.621677
2000-01-01 01:00:00,7377.478612,5548.286510,6388.840815,1463.841815,1049.928623,21828.376375
2000-01-01 02:00:00,6544.479503,5262.845013,5783.082886,1272.836862,1025.744617,19888.988882
2000-01-01 03:00:00,6060.659007,5028.138427,5218.125017,1129.042928,1098.926249,18534.891628
2000-01-01 04:00:00,5916.311304,4896.005504,4916.122613,1065.894377,1134.457590,17928.791389
...,...,...,...,...,...,...
2009-12-31 19:00:00,9486.363689,7428.037753,8138.697008,1866.051805,1201.165076,28120.315331
2009-12-31 20:00:00,9212.556001,7203.645840,7504.186113,1697.477863,1284.477292,26902.343109
2009-12-31 21:00:00,9402.438846,7047.507267,6940.459187,1573.695935,1223.507098,26187.608333


In [3]:
# compute dimensionless demand factor d_t
demand_max=df_demand['NEM'].max()
s_demand=df_demand['NEM'].copy()/demand_max
s_demand.describe()

count    87672.000000
mean         0.672161
std          0.092507
min          0.423928
25%          0.601167
50%          0.677592
75%          0.741519
max          1.000000
Name: NEM, dtype: float64

In [4]:
# load wind data
df_wind=pd.read_csv('master_wind.csv')
df_wind['AEST']=pd.to_datetime(df_wind['AEST'])
df_wind=df_wind.drop(['UTC'], axis=1)
df_wind=df_wind.set_index('AEST')
df_wind=df_wind.loc[df_demand.index]
df_wind

,NSA,ADE,SESA,NQ,CQ,SWQ,SEQ,NNS,NCEN,SWNSW,CAN,CVIC,NVIC,MEL,LV,TAS
AEST,,,,,,,,,,,,,,,,
2000-01-01 00:00:00,0.792552,0.900420,0.644210,0.403763,0.415507,0.483523,0.00603,0.374893,0.694667,0.626465,0.467790,0.724037,0.59372,0.633517,0.202095,0.484073
2000-01-01 01:00:00,0.766328,0.856365,0.580175,0.388541,0.371980,0.489350,0.00104,0.358743,0.700705,0.639255,0.381120,0.713380,0.50393,0.576187,0.212760,0.440013
2000-01-01 02:00:00,0.746908,0.793130,0.535135,0.373810,0.338767,0.494440,0.00429,0.356910,0.672210,0.665310,0.317895,0.685130,0.41002,0.521380,0.188085,0.382600
2000-01-01 03:00:00,0.725102,0.732025,0.510325,0.377066,0.321703,0.493473,0.04573,0.381003,0.642995,0.668455,0.271250,0.651423,0.32779,0.449697,0.145870,0.312883
2000-01-01 04:00:00,0.693614,0.682340,0.509980,0.391760,0.346560,0.512010,0.12400,0.417337,0.623270,0.651665,0.230670,0.596157,0.30469,0.400473,0.119965,0.252610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-12-31 19:00:00,0.586854,0.879255,0.746110,0.169711,0.461060,0.665647,0.32507,0.385037,0.111002,0.365740,0.263950,0.674490,0.67382,0.691197,0.408010,0.684970
2009-12-31 20:00:00,0.580710,0.817115,0.671125,0.178006,0.445983,0.635643,0.37279,0.441637,0.134435,0.457050,0.309555,0.466570,0.59930,0.489017,0.379415,0.764133
2009-12-31 21:00:00,0.598102,0.776105,0.591110,0.193590,0.441120,0.581197,0.39511,0.479270,0.181993,0.627105,0.320730,0.337643,0.55522,0.234010,0.598010,0.734753


In [5]:
# load PV data
df_pv=pd.read_csv('master_PV.csv')
df_pv['AEST']=pd.to_datetime(df_pv['AEST'])
df_pv=df_pv.drop(['UTC'], axis=1)
df_pv=df_pv.set_index('AEST')
df_pv=df_pv.loc[df_demand.index]
df_pv

,NSA,ADE,SESA,NQ,CQ,SWQ,SEQ,NNS,NCEN,SWNSW,CAN,CVIC,NVIC,MEL,LV,TAS
AEST,,,,,,,,,,,,,,,,
2000-01-01 00:00:00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000
2000-01-01 01:00:00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000
2000-01-01 02:00:00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000
2000-01-01 03:00:00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000
2000-01-01 04:00:00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-12-31 19:00:00,0.018883,0.022422,0.015444,0.0,0.0,0.0,0.0,0.0,0.0,0.001978,0.0,0.003944,0.0003,0.002967,0.001789,0.007822
2009-12-31 20:00:00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000
2009-12-31 21:00:00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000


In [9]:
# combine
zones=df_wind.columns

df_VRE=pd.concat([df_wind.rename(dict(zip(zones, [zone+'_wind' for zone in zones])), axis=1),
                      df_pv.rename(dict(zip(zones, [zone+'_pv' for zone in zones])), axis=1)], axis=1)
df_VRE.describe()

,NSA_wind,ADE_wind,SESA_wind,NQ_wind,CQ_wind,SWQ_wind,SEQ_wind,NNS_wind,NCEN_wind,SWNSW_wind,...,SEQ_pv,NNS_pv,NCEN_pv,SWNSW_pv,CAN_pv,CVIC_pv,NVIC_pv,MEL_pv,LV_pv,TAS_pv
count,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,...,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000
mean,0.487176,0.518950,0.482749,0.412358,0.399124,0.371925,0.298820,0.369941,0.410580,0.430320,...,0.223738,0.230826,0.224122,0.227533,0.221578,0.220851,0.213939,0.200579,0.191443,0.184123
std,0.250937,0.311564,0.294525,0.186893,0.209994,0.216614,0.232979,0.229998,0.264962,0.256515,...,0.292910,0.300694,0.294765,0.299780,0.302242,0.298624,0.293695,0.280950,0.271458,0.273811
min,0.000170,0.000000,0.000000,0.003431,0.000033,0.000023,0.000000,0.000073,0.000015,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.283203,0.239799,0.226274,0.268482,0.233753,0.195482,0.115597,0.175866,0.176217,0.215271,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.487286,0.532403,0.470042,0.412669,0.403292,0.361328,0.243690,0.352348,0.392946,0.422647,...,0.003167,0.004674,0.003822,0.004537,0.001356,0.001878,0.001367,0.001156,0.000933,0.000000
75%,0.689897,0.805126,0.736311,0.553285,0.558590,0.530006,0.435457,0.540617,0.625332,0.633184,...,0.476179,0.497844,0.476866,0.475440,0.440186,0.460181,0.428022,0.382439,0.353978,0.318978
max,0.990404,0.991690,0.991685,0.966683,0.980837,0.986573,0.991660,0.991663,0.991540,0.991620,...,0.921856,0.952050,0.945950,0.952181,0.986578,0.948800,0.957956,0.967111,0.982356,1.000000


In [10]:
# pickle
import pickle

file1 = open('s_demand.pkl', 'wb')

# dump information to that file
pickle.dump(s_demand, file1)

# close the file
file1.close()


file2 = open('df_VRE.pkl', 'wb')

# dump information to that file
pickle.dump(df_VRE, file2)

# close the file
file2.close()


